In [1]:
import numpy as np
import pandas as pd
import requests
import time
from matplotlib import pyplot as plt
from pprint import pprint


In [48]:
df=pd.read_csv('annual_wages.csv')
df_1=df[['COUNTRY','Country','Time','Unit Code','Value']]
df_1.columns=['Country_code','Country','Year','Currency','Annual_income']
income_df=df_1.loc[df_1['Year']==2019]
income_df=income_df.drop_duplicates(subset=['Country'])
income_df=income_df.reset_index(drop=True)

In [49]:
rate=("https://api.exchangeratesapi.io/history?start_at=2019-01-01&end_at=2019-12-31&base=USD")
json = requests.get(rate).json()
rate_df=pd.DataFrame.from_dict(json['rates'])
avg_rate=rate_df.mean(axis=1)
avg_rate=avg_rate.to_frame()
avg_rate['Currency']=avg_rate.index
avg_rate.columns=['Rate','Currency']

In [51]:
income1_df=income_df.merge(avg_rate,on='Currency')
income1_df['income_USD']=income1_df['Annual_income']/income1_df['Rate']
#income1_df.drop(['Year','Annual_income','Rate'],axis=1)
income1_df

,Country_code,Country,Year,Currency,Annual_income,Rate,income_USD
0,AUS,Australia,2019,AUD,8.360248e+04,1.439273,58086.613623
1,AUT,Austria,2019,EUR,4.468883e+04,0.893409,50020.596455
2,BEL,Belgium,2019,EUR,4.724438e+04,0.893409,52881.047124
3,FIN,Finland,2019,EUR,4.261754e+04,0.893409,47702.181469
4,FRA,France,2019,EUR,3.909926e+04,0.893409,43764.140735
5,DEU,Germany,2019,EUR,4.242128e+04,0.893409,47482.503953
6,GRC,Greece,2019,EUR,1.758946e+04,0.893409,19688.039846
7,IRL,Ireland,2019,EUR,4.960502e+04,0.893409,55523.327202
8,ITA,Italy,2019,EUR,3.002826e+04,0.893409,33610.894799
9,LUX,Luxembourg,2019,EUR,6.584320e+04,0.893409,73698.861923


In [74]:
region=requests.get('https://api.isthereanydeal.com/v01/web/regions/')
region_df=pd.DataFrame.from_dict(region.json()['data'])
region=list(region_df.columns) 


In [143]:
no=3
url = "https://api.isthereanydeal.com/v01/stats/popularity/chart/?"
key = "42c654a7eee83f02992f0317564462bceccdd585"
top_url = f"{url}key={key}&offset=0&&limit={no}"
response = requests.get(top_url).json()

In [144]:
top_df=pd.DataFrame.from_dict(response['data'])
top_df.drop(columns=['rank'])

,position,title,plain
0,1,NieR: Automata,nierautomata
1,2,Stardew Valley,stardewvalley
2,3,Divinity: Original Sin 2 - Definitive Edition,divinityoriginalsiniidefinitiveedition


In [145]:
price_df=pd.DataFrame()
currency_list=[]
for index in top_df.plain:
    for i in region:
        url_2="https://api.isthereanydeal.com/v01/game/prices/?"
        price_url=f"{url_2}key={key}&plains={index}&shops=steam&region={i}"
        response_1 = requests.get(price_url).json()        
        price_dict=response_1['data']
        currency_list.append(response_1['.meta']['currency'])
        price_df=price_df.append(price_dict[index]['list'],ignore_index=True)
        
price_df['currency']=currency_list   


In [135]:
response_1['.meta']

{'currency': 'CNY'}

In [136]:
response_1

{'.meta': {'currency': 'CNY'},
 'data': {'nierautomata': {'list': [{'price_new': 274,
     'price_old': 274,
     'price_cut': 0,
     'url': 'https://store.steampowered.com/app/524220/',
     'shop': {'id': 'steam', 'name': 'Steam'},
     'drm': ['Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r']}],
   'urls': {'game': 'https://isthereanydeal.com/#/page:game/info?plain=nierautomata'}}}}

In [139]:
currency_list

['EUR', 'EUR', 'GBP', 'USD', 'CAD', 'BRL', 'AUD', 'RUB', 'TRY', 'CNY']

In [146]:
price_df

,price_new,price_old,price_cut,url,shop,drm,currency
0,39.99,39.99,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],EUR
1,39.99,39.99,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],EUR
2,29.99,29.99,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],GBP
3,39.99,39.99,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],USD
4,53.50,53.50,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],CAD
5,107.00,107.00,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],BRL
6,47.50,47.50,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],AUD
7,1332.00,1332.00,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],RUB
8,133.00,133.00,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],TRY
9,274.00,274.00,0,https://store.steampowered.com/app/524220/,"{'id': 'steam', 'name': 'Steam'}",[Denuvo Anti-tamper\t\t\t\t\t\t\t\t\r],CNY
